# Import libraries

In [11]:
import numpy as np
import tensorflow as tf

# Compare models

## Trousers and Jeans

In [12]:
data_tj_test = np.load('dataset/Trousers & Jeans - All - Test.npz')

In [13]:
data_tj_test.files

['images', 'labels', 'genders']

In [42]:
np.unique(data_tj_test['labels'])

array([0, 1, 2, 3])

In [14]:
images_tj_test = data_tj_test['images']

genders_tj_test = data_tj_test['genders']
types_tj_test = data_tj_test['labels'] % 2

# The Type label is contained in the combined labels:
#    - 0 and 2 correspond to Trousers   (0)
#    - 1 and 3 correspond to Jeans      (1)
# 0 and 2 are both even, 1 and 3 are odd
# Therefore '% 2' works as it: 
#     returns 0, for input 0 and 2 (Trousers)
# and returns 1, for input 1 and 3 (Jeans)

In [15]:
# Standardized the images
images_tj_test = images_tj_test/255.0

In [16]:
# load the best trained models for each classification
model_tj_all = tf.keras.models.load_model('saved_models/Model 3 (Trousers&Jeans_All)/Run-62')

model_tj_gender = tf.keras.models.load_model('saved_models/Model 1 (Trousers&Jeans_Gender)/Run-12')
model_tj_male = tf.keras.models.load_model('saved_models/Model 1 (Trousers&Jeans_Male)/Run-5')
model_tj_female = tf.keras.models.load_model('saved_models/Model 1 (Trousers&Jeans_Female)/Run-13')

In [37]:
# In this case only accuracy won't be enough becuz we have 2 classes (genders and labels)
# but our 2 models have 2 different approaches; combined and hierarchical
# if we use only accuracy it will be hard to compare because hierarchical models have 3 models
# but combined model have only 1 model
# So, we have to define new measures
# in this case we give 1 score if model can predict Gender correctly, and 1 score if model can predict label correctly

score_tj_all = 0
score_tj_hierarchical = 0

In [38]:
#predict 1 by 1 image at a time
for i in range(len(images_tj_test)):
    
    #Test combined model
    
    # model require 4 dimensional input, if we use only [i] it will give (120,90,3)
    # so, we use [i:i+1] it will give (1,120,90,3)
    predict_tj_all = model_tj_all(images_tj_test[i:i+1])
    # the label is the index corresponding to the highest score
    label_tj_all = np.argmax(predict_tj_all)
    # 0 1 2//2=1 3//2=1
    if label_tj_all // 2 == genders_tj_test[i]: # Combined label // 2 correspond to 'gender' label
        score_tj_all += 1
    if label_tj_all % 2 == types_tj_test[i]: # Combined label % 2 correspond to 'type' label
        score_tj_all += 1
        
    #Test hierarchical model
    
    predict_tj_gender = model_tj_gender(images_tj_test[i:i+1])
    label_tj_gender = np.argmax(predict_tj_gender)
    
    if label_tj_gender == genders_tj_test[i]:
        score_tj_hierarchical += 1
    
    # If predict 'Male'
    if label_tj_gender == 0: 
        predict_tj_male = model_tj_male(images_tj_test[i:i+1])
        label_tj_type = np.argmax(predict_tj_male)
        
        if label_tj_type == types_tj_test[i]:
            score_tj_hierarchical += 1
            
    # If predict 'Female'
    if label_tj_gender == 1: 
        predict_tj_female = model_tj_male(images_tj_test[i:i+1])
        label_tj_type = np.argmax(predict_tj_female)
        
        if label_tj_type == types_tj_test[i]:
            score_tj_hierarchical += 1
            

In [39]:
# Show the final the scores
print(f"Combined Labels: \n{score_tj_all} points \n \nHierarchical Classification: \n{score_tj_hierarchical} points")

Combined Labels: 
869 points 
 
Hierarchical Classification: 
868 points


### There is no difference in performance for this work!

## Shoes

In [43]:
data_shoes_test = np.load('dataset/Shoes - All - Test.npz')

In [47]:
data_shoes_test.files

['images', 'labels', 'genders']

In [49]:
np.unique(data_shoes_test['labels'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [55]:
images_shoes_test = data_shoes_test['images']
genders_shoes_test = data_shoes_test['genders']
types_shoes_test = data_shoes_test['labels']

In [56]:
# standardize the images
images_shoes_test = images_shoes_test / 255.0

In [58]:
# load the best trained model
model_shoes_all = tf.keras.models.load_model('saved_models/Model 1 (Shoes_All_Dropout)/Run-18')

model_shoes_gender = tf.keras.models.load_model('saved_models/Model 1 (Shoes_Gender_L2)/Run-14')
model_shoes_male = tf.keras.models.load_model('saved_models/Model 1 (Shoes_Male_L2)/Run-2')
model_shoes_female = tf.keras.models.load_model('saved_models/Model 1 (Shoes_Female_L2)/Run-7')

In [78]:
# define the score counter
score_shoes_all = 0
score_shoes_hierarchical = 0

In [79]:
for i in range(len(images_shoes_test)):
    
    # combined model
    predict_shoes_all = model_shoes_all(images_shoes_test[i:i+1])
    predicted_label = np.argmax(predict_shoes_all)
    
#     all classes are 'Boots Male','Trainers/Sneakers Male',
#                     'Sandals/Flip flops/Slippers Male', 'Formal shoes Male',
#                     'Others Male',
#                     'Boots Female', 'Ballerina Female', 'Trainers/Sneakers Female',
#                     'High heels Female', 'Sandals/Flip flops/Slippers Female',
#                     'Others Female'
                        
    predicted_label_gender = (predicted_label >= 5) # The first 5 label are male and male is labels as 0
    if predicted_label_gender == genders_shoes_test[i]:
        score_shoes_all += 1
    
    if predicted_label == types_shoes_test[i]:
        score_shoes_all += 1
        
    # Hierarachical model
    predict_shoes_gender = model_shoes_gender(images_shoes_test[i:i+1])
    predicted_gender = np.argmax(predict_shoes_gender)
    
    if predicted_gender == genders_shoes_test[i]:
        score_shoes_hierarchical += 1
    
    # Male model
    if predicted_gender == 0:
        predict_shoes_male = model_shoes_male(images_shoes_test[i:i+1])
        predicted_male = np.argmax(predict_shoes_male)
        
        if predicted_male == types_shoes_test[i]:
            score_shoes_hierarchical += 1
    
    #female
    if predicted_gender == 1:
        predict_shoes_female = model_shoes_female(images_shoes_test[i:i+1])
        predicted_female = np.argmax(predict_shoes_female)
        
        # -5 because in the separated model, the model predict 0 to 5
        # But in the all gender they are 5 to 10 (remember first 5 (0 to 4) are male)
        if predicted_female == types_shoes_test[i]-5: 
            score_shoes_hierarchical += 1

In [80]:
print(f'Combined score: {score_shoes_all}\nHierarchical score: {score_shoes_hierarchical}')

Combined score: 981
Hierarchical score: 991


### For shoes, hierarchical model perform around 10% better than combined one. Good!